In [33]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from skimage.transform import resize
import matplotlib.pyplot as plt
from skimage.data import shepp_logan_phantom
import cv2

# Load matrix.csv and result.csv
print("now")
matrix = pd.read_csv('matrix_40d.csv', header=None).values
result = pd.read_csv('result_40d.csv', header=None).values
N, M = matrix.shape
print(N, M)
print(result.shape)
x = np.zeros(N)
print(x.shape)
phantom = shepp_logan_phantom()
phantom_resized = resize(phantom, (128, 128), mode='reflect', anti_aliasing=True)
phantom_vector = phantom_resized.flatten()
lr = 0.9
iterations = 30
directions = 40
img_n = 128
scaler = MinMaxScaler(feature_range=(0, 255))
img = np.zeros((iterations * directions, img_n, img_n), np.uint8)
img_3 = np.zeros((iterations * directions, img_n, img_n, 3), np.uint8)


now
16384 5120
(5120, 1)
(16384,)


/Users/fangxk2003/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/skimage/color/colorconv.py:984: RuntimeWarning: divide by zero encountered in matmul
  return rgb @ coeffs
/Users/fangxk2003/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/skimage/color/colorconv.py:984: RuntimeWarning: overflow encountered in matmul
  return rgb @ coeffs
/Users/fangxk2003/miniconda3/envs/jupyter_env/lib/python3.10/site-packages/skimage/color/colorconv.py:984: RuntimeWarning: invalid value encountered in matmul
  return rgb @ coeffs


In [34]:
x = np.zeros(N)
for i in range(iterations):
    for rays in range(M):
        sum_matrix = np.sum(np.square(matrix[:, rays]))
        x += lr * (result[rays] - np.dot(matrix[:, rays], x)) * matrix[:, rays] / sum_matrix
        if ((rays + 1) % img_n == 0) : 
            # truncate the image to 0-255
            xx = np.clip(x.reshape(128, 128), 0, 1) * 255
            img[i * directions + rays // img_n] = xx.astype(np.uint8)
    mse = np.mean(np.square(x - phantom_vector))
    print("MSE:", mse)

MSE: 0.011210862340536292
MSE: 0.007369004275568025
MSE: 0.00623420033433274
MSE: 0.005813688519921442
MSE: 0.0056619957013968845
MSE: 0.005598241708390754
MSE: 0.005564501668914695
MSE: 0.005539533530788237
MSE: 0.005521537817594035
MSE: 0.005507023048557699
MSE: 0.005495525842526367
MSE: 0.005485745923382711
MSE: 0.005477477956067993
MSE: 0.005470238402869715
MSE: 0.005463895228419817
MSE: 0.005458242540520259
MSE: 0.005453195841087518
MSE: 0.005448626938168959
MSE: 0.005444489640327888
MSE: 0.005440700736720072
MSE: 0.005437228709506374
MSE: 0.0054340222582729605
MSE: 0.005431056765400489
MSE: 0.0054283000512572455
MSE: 0.005425733379470854
MSE: 0.005423335176185112
MSE: 0.005421091568694665
MSE: 0.0054189871453897845
MSE: 0.00541701152427343
MSE: 0.005415153180598813


In [35]:
for i in range(iterations):
    for d in range(directions):
        # change img[i * directions + d] to 3 channels
        img_3[i * directions + d] = cv2.merge([img[i * directions + d]] * 3)
        cv2.putText(img_3[i * directions + d],  str(i) + "," + str(d), (5, 10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1)
out = cv2.VideoWriter('output_video.avi', cv2.VideoWriter_fourcc(*'MJPG'), 30, (img_n, img_n))
print(img_3.shape)
for frame in img_3:
    out.write(frame)
out.release()

(1200, 128, 128, 3)


In [36]:
cv2.imwrite('last_frame_40d.png', img_3[-1])

True